## How Much Does Brighter-Fatter Affect Object Shapes
### Plot shape difference between XX, YY measured moments and PSF XX, YY moments

Adapted by Michael Wood-Vasey on 2018-08-21  
from code from Merlin Fisher-Levine  
from code from Lauren MacArthur.

Runs on a pair of re-runs.  One with brighter-fatter correction applied.  The other without.


In [ ]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
%matplotlib inline

In [ ]:
def plot(mags, diffs, title='', saveFilename=''):
    maxDiff = 12
    nStdev = 3.0

    ptSize = 25
    fontsize = 20
    color = 'b'

    left, width = 0.12, 0.62
    bottom, height = 0.10, 0.85 # had to change height to make it save correctly
    left_h = left + width + 0.03
    bottom_h = bottom + width + 0.02
    rect_scatter = [left, bottom, width, height]
    rect_histy = [left_h, bottom, 0.20, height]

    deltaMin = 0.0

    fig = plt.figure(figsize=(15,10))
    axScatter = plt.axes(rect_scatter)
    if title: plt.title(title, fontsize=25)
    axHisty = plt.axes(rect_histy)

    _ = axScatter.scatter(mags, diffs, s=1.3*ptSize, marker="o", facecolors="b", edgecolors=color, alpha = 0.15)
    _ = axScatter.axhline(0, ls='--', color="0.4")

    axScatterY1 = axScatter.get_ylim()[0]
    axScatterY2 = axScatter.get_ylim()[1]
    nyDecimal = int(-1.0*np.around(np.log10(0.05*abs(axScatterY2 - (axScatterY1 - deltaMin))) - 0.5))
    yBinwidth = max(0.5/10**nyDecimal, np.around(0.02*abs(axScatterY2 - (axScatterY1 - deltaMin)), nyDecimal))

    yBins = np.arange((axScatterY1 - deltaMin) - 0.5*yBinwidth, axScatterY2 + 0.55*yBinwidth, yBinwidth)
    _ = axHisty.hist(diffs, bins=yBins, color='b', alpha=0.6, orientation="horizontal", label='test')

    _ = axScatter.set_ylim(-1.*maxDiff,maxDiff)
    _ = axHisty.set_ylim(-1.*maxDiff,maxDiff)
    axHisty.set_xscale("log", nonposy="clip")
    
    axScatter.set_xlabel('mag')
    axScatter.set_ylabel('Percentage difference star - model size: (sqrt(<xx+yy>/2)')
    
    mean = np.mean(diffs)
    std = np.std(diffs)
    std_clip = np.std( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    mean_clip = np.mean ( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    
    spc = 0.04
    _ = plt.text(0.05, 0.95-0*spc, "Star N       = %s"  %len(diffs), ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-1*spc, "stddev      = %.4f" %std,        ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-2*spc, "std_clip     = %.4f"%std_clip,   ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-3*spc, "mean        = %.4f" %mean,       ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-4*spc, "mean_clip = %.4f"   %mean_clip,  ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')

    if saveFilename:
        plt.savefig(saveFilename)


In [ ]:
# repo = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output'
# repo = '/global/homes/w/wmwv/DC2-production/Notebooks/bf_kernel/imsim_repo/rerun/no_bf_correction'
repo = '/global/homes/w/wmwv/DC2-production/Notebooks/bf_kernel/imsim_repo/rerun/with_bf_correction'

outdir = './plots'


try:
    os.makedirs(outdir)
except:
    pass

In [ ]:
run = os.path.basename(repo)
print(run)

In [ ]:
butler = dafPersist.Butler(repo)

In [ ]:
visits = [204595]
detectors = [0]

In [ ]:
from copy import copy

def get_shape_mag_diff(visits, partial_data_id):
    star_sizes = []
    model_sizes = []
    mags = []
    for visit in visits:
        data_id = copy(partial_data_id)
        data_id['visit'] = visit
        try:
            cat = butler.get('src', dataId=data_id)  
            calib = butler.get('calexp_calib', dataId=data_id)
        except dafPersist.NoResults as e:
            print(e)
            continue

        calib.setThrowOnNegativeFlux(False)

        mask = (cat['calib_psfUsed'] == 1) #& (cat['base_PixelFlags_flag_interpolated'] == 0)
        cat = cat[mask]
        star_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_xx'] + cat['base_SdssShape_yy']))])
        model_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_psf_xx'] + cat['base_SdssShape_psf_yy']))])
        mags.extend([_ for _ in calib.getMagnitude(cat['base_PsfFlux_flux'])])
        
    diffs = [100*(a - b)/b for a,b in zip(star_sizes, model_sizes)]
    diffs = np.asarray(diffs)

    return mags, diffs



In [ ]:
for detector in detectors:
    partial_data_id = {'detector': detector}
    mags, diffs = get_shape_mag_diff(visits, partial_data_id)

    if len(mags) < 1:
        print('No results for {}'.format(partial_data_id))
        continue

    title = '{}_detector_{}'.format(run, detector)
    saveName = os.path.join(outdir, title + '.png')
    plot(mags, diffs, title, saveName)  
    print('Finished detector: {}'.format(detector))
    sys.stdout.flush()    

In [ ]:
fails = []
for detector in detectors:
    for visit in visits:
        try:
            cat = butler.get('src', dataId={'detector': detector, 'visit': visit})
        except:
            fails.append((detector, visit))

In [ ]:
fails